# cypher query graphDB

In [1]:
#graphDB config
import os
import dotenv
dotenv.load_dotenv()

True

In [2]:
# import os
# import dotenv

# os.environ['NEO4J_URI']=os.getenv('NEO4J_URI')
# os.environ['NEO4J_USERNAME']=os.getenv('NEO4J_USERNAME')
# os.environ['NEO4J_PASSWORD']=os.getenv('NEO4J_PASSWORD')

In [3]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
                 )

In [4]:
dotenv.load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [5]:
# ! pip install langchain_openai

In [6]:
from langchain_groq import ChatGroq
from langchain_openai import AzureChatOpenAI
# llm=ChatGroq(
#     groq_api_key=groq_api_key,
#     model_name='Gemma2-9b-it'
#     )
llm=AzureChatOpenAI(o
    azure_deployment=s.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0.1,

)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000016EC2ADC5B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000016EC2AFE730>, root_client=<openai.lib.azure.AzureOpenAI object at 0x0000016EBBFB1130>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x0000016EC2ADC610>, temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://easttest123.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview', deployment_name='gpt-35-turbo', openai_api_version='2024-02-15-preview', openai_api_type='azure')

In [7]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk(born June 28, 1971) is a businessman and investor known 
for his key roles in the space company SpaceX and the automotive company 
Tesla, Inc. Other involvements include ownership of X Corp., the company 
that operates the social media platform X (formerly known as Twitter), 
and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. 
He is one of the wealthiest individuals in the world; as of August 2024 
Forbes estimates his net worth to be US$247 billion. Musk was born in Pretoria, 
South Africa, to Maye (née Haldeman), a model, and Errol Musk, a businessman 
and engineer. Musk briefly attended the University of Pretoria before immigrating 
to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. 
"""
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content='\nElon Reeve Musk(born June 28, 1971) is a businessman and investor known \nfor his key roles in the space company SpaceX and the automotive company \nTesla, Inc. Other involvements include ownership of X Corp., the company \nthat operates the social media platform X (formerly known as Twitter), \nand his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. \nHe is one of the wealthiest individuals in the world; as of August 2024 \nForbes estimates his net worth to be US$247 billion. Musk was born in Pretoria, \nSouth Africa, to Maye (née Haldeman), a model, and Errol Musk, a businessman \nand engineer. Musk briefly attended the University of Pretoria before immigrating \nto Canada at the age of 18, acquiring citizenship through his Canadian-born mother. \n')]

In [8]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [9]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [10]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Spacex', type='Company', properties={}), Node(id='Tesla, Inc.', type='Company', properties={}), Node(id='X Corp.', type='Company', properties={}), Node(id='X', type='Social media platform', properties={}), Node(id='Twitter', type='Social media platform', properties={}), Node(id='Boring Company', type='Company', properties={}), Node(id='Xai', type='Company', properties={}), Node(id='Neuralink', type='Company', properties={}), Node(id='Openai', type='Company', properties={}), Node(id='Pretoria', type='City', properties={}), Node(id='South Africa', type='Country', properties={}), Node(id='Maye Haldeman', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='University Of Pretoria', type='University', properties={}), Node(id='Canada', type='Country', properties={})], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), targe

In [11]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Spacex', type='Company', properties={}),
 Node(id='Tesla, Inc.', type='Company', properties={}),
 Node(id='X Corp.', type='Company', properties={}),
 Node(id='X', type='Social media platform', properties={}),
 Node(id='Twitter', type='Social media platform', properties={}),
 Node(id='Boring Company', type='Company', properties={}),
 Node(id='Xai', type='Company', properties={}),
 Node(id='Neuralink', type='Company', properties={}),
 Node(id='Openai', type='Company', properties={}),
 Node(id='Pretoria', type='City', properties={}),
 Node(id='South Africa', type='Country', properties={}),
 Node(id='Maye Haldeman', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='University Of Pretoria', type='University', properties={}),
 Node(id='Canada', type='Country', properties={})]

In [12]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Company', properties={}), type='KEY_ROLE', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Company', properties={}), type='KEY_ROLE', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Company', properties={}), type='OWNERSHIP', properties={}),
 Relationship(source=Node(id='X Corp.', type='Company', properties={}), target=Node(id='X', type='Social media platform', properties={}), type='OPERATES', properties={}),
 Relationship(source=Node(id='X Corp.', type='Company', properties={}), target=Node(id='Twitter', type='Social media platform', properties={}), type='OPERATES', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Boring Company', type='Company', properties

# load dataset to graphDB and generating cypher queries

In [13]:
# load the movie dataset 

movie_query=""" 
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released=date(row.released),
    m.title=row.title,
    m.imdbRating=tofloat(row.imdbRating)
FOREACH (director in split(row.director,'|')|
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]-> (m)
         )
FOREACH (actor in split(row.actors,'|')|
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]-> (m)
        )
FOREACH (genre in split(row.genres,'|')|
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
        )

"""

In [14]:
graph

In [15]:
# insert records 
graph.query(movie_query)

[]

In [16]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


# create cypher query using llm model

In [17]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000016EBC08EE50>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000016EC2ADC5B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000016EC2AFE730>, 

In [18]:
response=chain.invoke({'query':'give me the names of actors and movies whose movies have imdb rating greater than 5'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.imdbRating > 5
RETURN p.name, m.title
Full Context:
[{'p.name': 'Jim Varney', 'm.title': 'Toy Story'}, {'p.name': 'Tim Allen', 'm.title': 'Toy Story'}, {'p.name': 'Tom Hanks', 'm.title': 'Toy Story'}, {'p.name': 'Don Rickles', 'm.title': 'Toy Story'}, {'p.name': 'Robin Williams', 'm.title': 'Jumanji'}, {'p.name': 'Bradley Pierce', 'm.title': 'Jumanji'}, {'p.name': 'Kirsten Dunst', 'm.title': 'Jumanji'}, {'p.name': 'Jonathan Hyde', 'm.title': 'Jumanji'}, {'p.name': 'Walter Matthau', 'm.title': 'Grumpier Old Men'}, {'p.name': 'Ann-Margret', 'm.title': 'Grumpier Old Men'}]

> Finished chain.


{'query': 'give me the names of actors and movies whose movies have imdb rating greater than 5',
 'result': 'The actors and movies with IMDb ratings greater than 5 are:\n- Jim Varney in Toy Story\n- Tim Allen in Toy Story\n- Tom Hanks in Toy Story\n- Don Rickles in Toy Story\n- Robin Williams in Jumanji\n- Bradley Pierce in Jumanji\n- Kirsten Dunst in Jumanji\n- Jonathan Hyde in Jumanji\n- Walter Matthau in Grumpier Old Men\n- Ann-Margret in Grumpier Old Men'}